In [ ]:
import cv2
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt


In [ ]:
#read images
#source,0--->for gray scale
#you can use ur own paths here
img1=cv2.imread("C:/Users/prana/OneDrive/Desktop/projects/OPEN-CV/training_data/20200313093834_IMG_9535.jpg",0)
img2=cv2.imread("C:/Users/prana/OneDrive/Desktop/projects/OPEN-CV/training_data/20200313093934_IMG_9536.jpg",0)
img1=cv2.resize(img1,(600,600))
img2=cv2.resize(img2,(600,600))


cv2.imshow("Image1",img1)
cv2.imshow("Image2",img2)
cv2.waitKey(0)

In [ ]:
#creating detector ORB-->oriented fast and rotated brief
#it defaultly detects 500 features generally for any image
#ORB uses a modified version of fast algorithm called oriented fast algorithm
#ORB is free and fast version else swift/surf detectors are patented and not free

orb=cv2.ORB_create(nfeatures=1000) #if nothing mentioned its 500

#kp's are the keypoints to identify a specific feature
#des's are the descriptors --->describing the image in terms of numbers array-->(500,32) always size
#i,e 500 features default and each feature is described with 32 values

kp1,des1=orb.detectAndCompute(img1,None) #image and mask
kp2,des2=orb.detectAndCompute(img2,None) #image and mask

#drawing keypoints detected

imgKp1=cv2.drawKeypoints(img1,kp1,None)
imgKp2=cv2.drawKeypoints(img2,kp2,None)

cv2.imshow("Image1 with keypoints",imgKp1)
cv2.imshow("Image2 with keypoints2",imgKp2)
cv2.waitKey(0)

In [ ]:
#using matchers to match the descriptors as both are of the particularly same images
#matchers match every des of one to other
#the other technique is k nearest technique-->here used

bf=cv2.BFMatcher()
matches=bf.knnMatch(des1,des2,k=2) #k=2-->2 values to compare

#deciding whether a match is a good match/not
good_match=[]
for i,j in matches :
    if i.distance<0.75*j.distance :
        good_match.append([i])#append if distance is less
print(len(good_match))
imgMatches=cv2.drawMatchesKnn(img1,kp1,img2,kp2,good_match,None,flags=2)
cv2.imshow("Matches",imgMatches)
cv2.waitKey(0)

In [ ]:
#image classifier
#using feature detectors
#generic code for clases and each image
#we use os-lib for this

#storing path for each image
path='C:/Users/prana/OneDrive/Desktop/projects/OPEN-CV/training_data'
#to classify images w.r.to classes 2 lists
images=[]
classnames=[]
mylist=os.listdir(path)
#print(mylist) #gives images names ---> always give a unique image for a specific identification
#hence len(mylist) gives num,ber of classes

for i in mylist :
    imgCurr=cv2.imread(path+'/'+i,0)
    imgCurr=cv2.resize(imgCurr,(500,500))
    images.append(imgCurr)
    classnames.append(i.split('.')[0])
#print(classnames)

#function takes list of images and creates descriptors for eaxh and store them
orb=cv2.ORB_create(nfeatures=1000) #if nothing mentioned its 500
kpList=[]
def findDes(images) :
    desList=[]
    for i in images :
        kp,des=orb.detectAndCompute(i,None)
        desList.append(des)
        kpList.append(kp)
    return desList

desList=findDes(images)
#print(len(desList)) #--->number of images

#function to find des for the captured image
def findID(img,desList) :
    kp2,des2=orb.detectAndCompute(img,None)
    #finding matches
    matchList=[] #for storing all achieving good matches
    
    finalval=-1 #for the label to be printed
    
    bf=cv2.BFMatcher()
    
    #matching each descriptor from desList to des2
    try:
        for des1 in desList :
    
            matches=bf.knnMatch(des1,des2,k=2) #k=2-->2 values to compare
            #deciding whether a match is a good match/not
            good_match=[]
            for i,j in matches :
                if i.distance<0.75*j.distance :
                    good_match.append([i])#append if distance is less
            #imgMatches=cv2.drawMatchesKnn(images[desList.index(des1)],kpList[desList.index(des1)],img,kp2,good_match,None,flags=2)
            #cv2.show("matching",imgMatches)
            #cv2.waitKey(0)
            matchList.append(len(good_match)) #no of good matches for each of the images
        print(matchList)
    #try except-->if no match
    except :
        pass
    threashhold=5 #usual threashold
    if len(matchList)!=0 :
        if max(matchList)>=threashhold :
            finalval=matchList.index(max(matchList))
    return finalval
#--------------->TESTING<----------------#        
#getting camera feed to detect
#cap=cv2.VideoCapture(0)

#defining while to read images from webcam
#while True :
#    success,img2=cap.read()
#    imgOriginal=img2.copy()
#    img2=cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)
#    cv2.imshow("Image",img2)
#    cv2.waitKey(0)
#not using webcam because of webcam troubles
        
#storing path for each image
path='C:/Users/prana/OneDrive/Desktop/projects/OPEN-CV/testing_data'
#to classify images w.r.to classes 2 lists
mylist=os.listdir(path)
#print(mylist) #gives images names ---> always give a unique image for a specific identification
#hence len(mylist) gives num,ber of classes
print(mylist)
c=0
for i in mylist :
    imgCurr=cv2.imread(path+'/'+i,0)
    imgCurr=cv2.resize(imgCurr,(500,500))
    c+=1
    print("Image number",c)
    index=findID(imgCurr,desList)
    if index!=-1 :
        print(index)
        cv2.putText(imgCurr,classnames[index],(50,50),cv2.FONT_HERSHEY_COMPLEX,1,(255,0,255),1)
    else :
        print(index)
        cv2.putText(imgCurr,"Not detected",(50,50),cv2.FONT_HERSHEY_COMPLEX,1,(255,0,255),1)
    cv2.imshow("DETECTION",imgCurr)
    #plt.imshow(imgCurr,cmap='gray')
    print()
    cv2.waitKey(0)